In [ ]:
import cv2
import numpy as np
import json
import os
import requests
from datetime import datetime
from shapely.geometry import Polygon, Point

# Load annotations json file for frame detections
file_path = "annotations.json"  # Update with actual path of the json file
with open(file_path, "r") as file:
    data = json.load(file)

# Definig the frame structure fetched from the json file
frames_data = data["frames"]

# Define the given canvas size
WIDTH, HEIGHT = 1920, 1080

# Define the given ROIs (Region of Interests)
ROI_1 = [(885, 85), (834, 246), (1228, 260), (1139, 77)]
ROI_2 = [(181, 288), (165, 522), (612, 510), (544, 246)]

# Creating polygon shapely geometrical objects for the given ROIs using Shapely geometric libraries
polygon_roi_1 = Polygon(ROI_1)
polygon_roi_2 = Polygon(ROI_2)

# Define colors for different object classes
COLORS = {"Person": (0, 255, 0), "Car": (255, 0, 0), "Truck": (0, 0, 255)}

# Slack Webhook URL of slack channel 
SLACK_WEBHOOK_URL = ""

        
# Function to send event to Slack
def send_slack_alert(timestamp, rule_name):
    message = {
        "text": f"**Event Detected**:\nTimestamp: {timestamp}\nRule: {rule_name}"
    }
    response = requests.post(SLACK_WEBHOOK_URL, json=message)
    print(f"Slack response: {response.status_code}, {response.text}")
    if response.status_code != 200:
        print(f"Error sending Slack message: {response.status_code}, {response.text}")
    else:
        print("Event alert sent to Slack")

# Video writer setup for 5 frames per second by defining the video codec for mp4v encoding
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 5, (WIDTH, HEIGHT))

# Looping into the frame data fetched from the annotations.json file
for frame in frames_data:
    # Drawing the black canvas for drawing the ROIs and checking detections
    canvas = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
    
    # Checking the Car and person inside ROIs by creating a Dictionary of two given ROIs and initializing the objects as zero
    objects_in_roi_1 = {"Person": 0, "Car": 0}
    objects_in_roi_2 = {"Person": 0, "Car": 0}
    
    # Drawing two given ROIs on the black canvas 
    cv2.polylines(canvas, [np.array(ROI_1, np.int32)], isClosed=True, color=(0, 255, 255), thickness=2)
    cv2.polylines(canvas, [np.array(ROI_2, np.int32)], isClosed=True, color=(255, 255, 0), thickness=2)
    
    # Now testing the detections
    for detection in frame["detections"]:
        class_label = detection["class"].title() #Class_label represents Car,Person and Truck
        bbox = detection["bbox"]  # [x, y, width, height] in fractional format --> left,top, height, width
        #bbox is the bounding box which is used to define object's i.e Class Label position and size
        
        
        # Converting fractional bbox to absolute pixel values of bbox in terms of [x,y] coordinates
        x1 = int(bbox["left"] * WIDTH)
        y1 = int(bbox["top"] * HEIGHT)
        x2 = int((bbox["left"] + bbox["width"]) * WIDTH)
        y2 = int((bbox["top"] + bbox["height"]) * HEIGHT)
        
        # Calculating the center of bounding box
        x_center = (x1 + x2)//2
        y_center = (y1 + y2)//2
        

        
        # Drawing bounding box of all the objects
        color = COLORS.get(class_label, (255, 255, 255))
        cv2.rectangle(canvas, (x1, y1), (x2, y2), color, 2)
        cv2.putText(canvas, class_label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Creating the point object at the centre of bounding box
        point = Point(x_center, y_center)
        
        #The below condition checks determines that if the centre of bounding box objects lies within ROIs then increment that
        #given class_label or object by 1 for Both ROI_1 and ROI_2
        if point.within(polygon_roi_1):
            objects_in_roi_1[class_label] = objects_in_roi_1.get(class_label, 0) + 1
            #print(f"Detected {class_label} in ROI 1")
            
        if point.within(polygon_roi_2):
            objects_in_roi_2[class_label] = objects_in_roi_2.get(class_label, 0) + 1
            #print(f"Detected {class_label} in ROI 2")
    
    # Printing the objects detected in ROI_1 and ROI_2
    print(f"ROI 1 Objects: {objects_in_roi_1}")
    print(f"ROI 2 Objects: {objects_in_roi_2}")


    
    # Check for events for both ROI_1 and ROI_2 to determine Person and Car is present in ROI at the same time
    event_detected_1 = objects_in_roi_1["Person"] > 0 and objects_in_roi_1["Car"] > 0
    event_detected_2 = objects_in_roi_2["Person"] > 0 and objects_in_roi_2["Car"] > 0
    
    
    # If one of the above conditions are satisfied then deepen the color of that ROI region, Calculate time stamp
    # and send Slack message alert to Slack Channel by calling send_slack_alert() function
    if event_detected_1:
        cv2.polylines(canvas, [np.array(ROI_1, np.int32)], isClosed=True, color=(0, 0, 255), thickness=3)
        print("Event detected in ROI 1")
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        send_slack_alert(timestamp, "Person and Car in ROI 1")
    
    if event_detected_2:
        cv2.polylines(canvas, [np.array(ROI_2, np.int32)], isClosed=True, color=(0, 0, 255), thickness=3)
        print("Event detected in ROI 2")
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        send_slack_alert(timestamp, "Person and Car in ROI 2")
    
    # Write frame to video
    out.write(canvas)

# Closing the Video writer
out.release()
print("Video generation : Output.mp4")


ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person': 0, 'Car': 0}
ROI 1 Objects: {'Person': 0, 'Car': 0}
ROI 2 Objects: {'Person':